In [ ]:
!pip install boto3

In [ ]:
import utils as u
import os

In [ ]:
if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
workbookDir

In [ ]:
aws_separator = "/"

In [ ]:
# Import S3 connectors librairies
#from awsPy.aws_authorization import aws_connector
import importlib.util
spec = importlib.util.spec_from_file_location("aws_connector.py", "C:/Users/Hp/Documents/GitHub/aws-python/awsPy/aws_authorization/aws_connector.py")
aws_connector = importlib.util.module_from_spec(spec)
spec.loader.exec_module(aws_connector)

#from awsPy.aws_s3 import service_s3
import importlib.util
spec = importlib.util.spec_from_file_location("service_s3.py", "C:/Users/Hp/Documents/GitHub/aws-python/awsPy/aws_s3/service_s3.py")
service_s3 = importlib.util.module_from_spec(spec)
spec.loader.exec_module(service_s3)

In [ ]:
# Connect to S3
import os
from pathlib import Path

bucket = 'calfdata'
path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)

con = aws_connector.aws_instantiate(credential = path_cred,
                                        region = 'eu-west-3')
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = 'calfdata')

In [ ]:
# Send Raw files to S3

In [ ]:
# List files to send and compute each destination
import os
prep_path= 'stock' #os.path.join(workbookDir,'flux')
root_path=aws_separator.join(['INPI', 'TC_1' , '00_RawData', 'public' ,'IMR_Donnees_Saisies' ,'tc'])

In [ ]:
list_files=[]
for root,dirs,files in os.walk(prep_path):
    for i, file in enumerate(files):
          if file.endswith(('.csv','.zip')):
                #print(root)
                
                source_full_path=os.path.join(root,file)
                #print(source_full_path)
                
                dest_path = aws_separator.join([root_path,root.replace("\\","/")])
                #print(dest_path)
                list_files.append((source_full_path,dest_path))
list_files[:5]

In [ ]:
# Send files to s3 (raw stock)

for (full_path,destination_in_s3) in list_files:
    filename=full_path.split('\\')[-1]
    key = aws_separator.join([destination_in_s3,filename])
    if not s3.key_exist(key):
        s3.upload_file(full_path, destination_in_s3)